In [1]:
%%html
<script>
(function() {
  // Create the toggle button
  const rtlButton = document.createElement("button");
  rtlButton.textContent = "Toggle LTR";
  rtlButton.id = "top-rtl-toggle";
  rtlButton.style.marginLeft = "8px";
  rtlButton.style.padding = "4px 10px";
  rtlButton.style.fontSize = "14px";
  rtlButton.style.cursor = "pointer";

  // State
  var rtlActive = false;

  // Styling function
  var applyStyleToEditor = (editor) => {
    if (!editor) return;
    var direction = getComputedStyle(editor).getPropertyValue('direction')=='rtl' ? 'ltr' : 'rtl';
    var text_align = getComputedStyle(editor).getPropertyValue('text-align')=='right' ? 'left' : 'right';
    editor.style.setProperty('direction', direction, 'important');
    editor.style.setProperty('text-align', text_align, 'important');
  };

  // Toggle logic
  rtlButton.onclick = () => {
    rtlActive = !rtlActive;
    rtlButton.textContent = rtlActive ? "Toggle LTR" : "Toggle RTL";
    document.querySelectorAll('.jp-MarkdownCell .jp-InputArea-editor').forEach(applyStyleToEditor);
    document.querySelectorAll('.jp-RenderedHTMLCommon code, .jp-RenderedHTMLCommon code span').forEach(applyStyleToEditor);
    document.querySelectorAll('jp-RenderedHTMLCommon, .jp-RenderedHTMLCommon *').forEach(applyStyleToEditor);
  };

  // Watch for focus into editing Markdown cells
  // document.addEventListener('focusin', (event) => {
  //   const editor = event.target.closest('.jp-MarkdownCell .jp-InputArea-editor');
  //    if (editor) applyStyleToEditor(editor);
  // });

  // Insert into top toolbar if not already present
  var insertIntoToolbar = () => {
    const toolbar = document.querySelector('.jp-NotebookPanel-toolbar');
    if (toolbar && !document.getElementById("top-rtl-toggle")) {
      toolbar.appendChild(rtlButton);
    } else {
      // Try again in a moment if toolbar isn't ready yet
      setTimeout(insertIntoToolbar, 300);
    }
  };

  insertIntoToolbar();
})();
</script>

In [2]:
%%html
<!-- <style>
  table {display: inline-block}
</style> -->

## טעינת ושמירת נתונים (I/O)

עבודה יעילה עם נתונים מתחילה בטעינה נכונה של הקבצים ושמירה מסודרת של תוצאות הביניים.  
Pandas מספקת ממשק נוח לקריאה וכתיבה של קבצי CSV ו־Parquet, לצד אפשרויות שליטה חשובות על סוגי הנתונים, פרשנות תאריכים וערכים חסרים.

#### קריאה: `read_csv`, `read_parquet`
- `read_csv` מתאימה לקבצים טקסטואליים מופרדים בפסיקים (או מפרידים אחרים).  
- `read_parquet` מיועדת לפורמט עמודות דחוס ומהיר ל־Analytics; כולל שמירה של מטא־דאטה על טיפוסים.

#### שמירה: `to_csv`, `to_parquet`
- `to_csv` שומרת טבלה לקובץ CSV (טקסט); קריא לבני־אדם, אך פחות יעיל לגודל ומהירות.  
- `to_parquet` שומרת לקובץ Parquet (עמודות; לעיתים דחוס יותר ומהיר לטעינה).

#### אפשרויות שימושיות: `usecols`, `dtype`, `parse_dates`, `na_values`
- `usecols` — טעינת תת־קבוצה של עמודות בלבד לחיסכון בזיכרון ובמהירות.  
- `dtype` — אכיפה/הגדרה מפורשת של טיפוסי הנתונים בעמודות (למשל `string`, `Int64`, `category`).  
- `parse_dates` — המרה אוטומטית של עמודות תאריך־ושעה לאובייקטי `datetime64[ns]`.  
- `na_values` — הגדרה מפורשת של ייצוגי "חסר" בטקסט (כגון `"NA"`, `"n/a"`, `"?"`) שיזוהו כ־NaN.


### תרגילים: טעינת נתוני ניסוי וסינון מדידות  

בקטע הבא נתרגל טעינת נתונים ניסיוניים מתוך קובץ **CSV**, המרת תאריכים, אכיפת טיפוסים מתאימים, ולבסוף שמירת תת־קבוצה מסוננת לדיסק.  

לפני שנקרא קובץ אמיתי, ניצור דוגמת נתונים קטנה ישירות מתוך הקוד —  
כך נוכל להדגים את יכולות הקריאה (`read_csv`) בצורה עצמאית, בלי צורך בקובץ שמור במערכת הקבצים.  

נשתמש במחרוזת טקסט (multi-line string) המדמה תוכן של קובץ **CSV** עם מידע על מדידות מניסוי פיזיקלי:  
- מזהה מדידה (`measurement_id`)  
- זמן מדידה (`timestamp`)  
- מתח (V), זרם (A), טמפרטורה (°C)  
- והערות נוספות (`notes`)  
 

In [3]:
# --- Setup: create a tiny "experiment" CSV on the fly (for a self-contained example) ---
import pandas as pd
from io import StringIO
import os

csv_text = """measurement_id,timestamp,voltage,current,temperature,notes
1,2025-01-03, 2.5, 0.10, 298.5, -
2,2025-01-05, 5.0, 0.20, 300.1, NA
3,2025/01/06,10.0, 0.50, 302.0, n/a
4,03-01-2025, 4.5, 0.15, 297.8, ?
5,2025-01-07,12.0,     , 305.0, -
"""

csv_buffer = StringIO(csv_text)

#### שלב 2: טעינת הנתונים עם `read_csv`

נשתמש בפונקציה `pd.read_csv()` עם מספר פרמטרים שימושיים:

- **`usecols`** — טעינת עמודות נבחרות בלבד.  
- **`dtype`** — הגדרת טיפוסי נתונים מתאימים לעמודות.  
- **`parse_dates`** — המרת עמודת הזמן לפורמט `datetime`.  
- **`na_values`** — הגדרת ערכים חסרים כגון `"NA"`, `"?"`, `"-"`.  


In [4]:
# --- Load: read_csv with useful options ---
measurements = pd.read_csv(
    csv_buffer,
    usecols=["measurement_id", "timestamp", "voltage", "current", "temperature", "notes"],
    dtype={
        "measurement_id": "Int64",
        "notes": "string"
    },
    parse_dates=["timestamp"],
    na_values=["-", "NA", "n/a", "?"],
    dayfirst=False
)

# Handle mixed date formats, if any
if measurements["timestamp"].dtype == "object":
    measurements["timestamp"] = pd.to_datetime(measurements["timestamp"], errors="coerce", infer_datetime_format=True)

# Convert numeric columns safely
for col in ["voltage", "current", "temperature"]:
    measurements[col] = pd.to_numeric(measurements[col], errors="coerce")

print("DataFrame loaded:")
display(measurements)


DataFrame loaded:


/tmp/ipykernel_20060/322451142.py:16: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  measurements["timestamp"] = pd.to_datetime(measurements["timestamp"], errors="coerce", infer_datetime_format=True)


,measurement_id,timestamp,voltage,current,temperature,notes
0,1,2025-01-03,2.5,0.10,298.5,-
1,2,2025-01-05,5.0,0.20,300.1,NA
2,3,NaT,10.0,0.50,302.0,n/a
3,4,NaT,4.5,0.15,297.8,?
4,5,2025-01-07,12.0,NaN,305.0,-


#### שלב 3: בדיקת מבנה הנתונים

נבדוק את מבנה הנתונים באמצעות `info()` ו־`describe()` כדי לוודא שכל עמודה קיבלה את הטיפוס המתאים ושאין ערכים חריגים.

In [5]:
print("\nInfo:")
measurements.info()

print("\nDescribe (numeric):")
display(measurements.describe())

print("\nDtypes:")
print(measurements.dtypes)



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   measurement_id  5 non-null      Int64         
 1   timestamp       3 non-null      datetime64[ns]
 2   voltage         5 non-null      float64       
 3   current         4 non-null      float64       
 4   temperature     5 non-null      float64       
 5   notes           5 non-null      string        
dtypes: Int64(1), datetime64[ns](1), float64(3), string(1)
memory usage: 377.0 bytes

Describe (numeric):


,measurement_id,timestamp,voltage,current,temperature
count,5.0,3,5.000000,4.000000,5.000000
mean,3.0,2025-01-05 00:00:00,6.800000,0.237500,300.680000
min,1.0,2025-01-03 00:00:00,2.500000,0.100000,297.800000
25%,2.0,2025-01-04 00:00:00,4.500000,0.137500,298.500000
50%,3.0,2025-01-05 00:00:00,5.000000,0.175000,300.100000
75%,4.0,2025-01-06 00:00:00,10.000000,0.275000,302.000000
max,5.0,2025-01-07 00:00:00,12.000000,0.500000,305.000000
std,1.581139,NaN,4.009364,0.179699,2.906372



Dtypes:
measurement_id             Int64
timestamp         datetime64[ns]
voltage                  float64
current                  float64
temperature              float64
notes             string[python]
dtype: object


#### שלב 4: סינון תוצאות לפי תנאים פיזיקליים

נניח שנרצה לנתח רק את המדידות שבהן המתח מעל 4 וולט
והזרם אינו חסר (נמדד בפועל).
נשתמש לשם כך בפונקציה `query()` של Pandas, שמאפשרת ניסוח תנאים באופן קריא:

In [6]:
# --- Filter: keep only valid physical measurements ---
filtered = measurements.query("voltage > 4 and current.notna()")

print("Filtered measurements:")
display(filtered)


Filtered measurements:


,measurement_id,timestamp,voltage,current,temperature,notes
1,2,2025-01-05,5.0,0.20,300.1,NA
2,3,NaT,10.0,0.50,302.0,n/a
3,4,NaT,4.5,0.15,297.8,?


### שלב 5: שמירת תוצאות לקבצים

נשמור את התוצאות לשני פורמטים:

- **CSV** – קובץ טקסט פשוט, נוח לקריאה.  
- **Parquet** – פורמט עמודות יעיל ודחוס, שומר מידע על טיפוסים ומאפשר טעינה מהירה.  

לאחר מכן נקרא מחדש את קובץ ה־Parquet כדי לוודא שהנתונים נשמרו בהצלחה.


In [7]:
# --- Save: write both CSV and Parquet outputs ---
os.makedirs("data", exist_ok=True)

csv_path = "data/filtered_measurements.csv"
parquet_path = "data/filtered_measurements.parquet"

filtered.to_csv(csv_path, index=False)
filtered.to_parquet(parquet_path, index=False)

print("Saved files:")
print("-", csv_path)
print("-", parquet_path)

# Optional: read back to verify
reloaded = pd.read_parquet(parquet_path)
print("\nReloaded from Parquet:")
display(reloaded)


Saved files:
- data/filtered_measurements.csv
- data/filtered_measurements.parquet

Reloaded from Parquet:


,measurement_id,timestamp,voltage,current,temperature,notes
0,2,2025-01-05,5.0,0.20,300.1,NA
1,3,NaT,10.0,0.50,302.0,n/a
2,4,NaT,4.5,0.15,297.8,?


`````{admonition} סיכום
:class: tip
בקטע זה למדנו:
- כיצד לטעון נתונים ניסיוניים ממבנה CSV ולזהות ערכים חסרים.  
- איך להמיר עמודות זמן וערכים נומריים לטיפוסים מתאימים.  
- כיצד לסנן נתונים לפי תנאים פיזיקליים בעזרת `query()`.  
- ולבסוף — איך לשמור תוצאות מעובדות לקבצי CSV ו־Parquet לשימוש עתידי.  
`````

In [8]:
import json
from jupyterquiz import display_quiz

quiz_json = \
'''
[{
  "question": "בעת טעינת נתוני ניסוי באמצעות הפונקציה <code>pd.read_csv()</code>, נרצה להבטיח שעמודת זמן המדידה תפורש כראוי ועמודות טקסט (כמו הערות) יישמרו בפורמט נכון.<br><br>איזה שילוב של פרמטרים מתאים לכך?",
  "type": "many_choice",
  "answers": [
    {
      "answer": "parse_dates=['timestamp'], dtype={'notes': 'string', 'experiment_id': 'string'}",
      "correct": true,
      "feedback": "נכון! הפרמטר parse_dates ממיר את עמודת הזמן לפורמט datetime64, ו־dtype מאפשר להגדיר במפורש את סוג הנתונים בעמודות טקסט."
    },
    {
      "answer": "convert_dates=True, data_type='string'",
      "correct": false,
      "feedback": "לא — אלו אינם פרמטרים תקפים ב־read_csv. Pandas משתמשת ב־parse_dates ו־dtype."
    },
    {
      "answer": "read_as_text=True, cast_all='object'",
      "correct": false,
      "feedback": "לא נכון — Pandas לא תומכת בפרמטרים כאלה לטעינת CSV."
    },
    {
      "answer": "parse_time=True, types={'timestamp': 'datetime'}",
      "correct": false,
      "feedback": "לא מדויק — אין פרמטר parse_time ב־read_csv. יש להשתמש ב־parse_dates."
    }
  ]
}]
'''
myquiz = json.loads(quiz_json)
display_quiz(myquiz)


<IPython.core.display.Javascript object>